```
--- Day 11: Seating System ---
Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (.), an empty seat (L), or an occupied seat (#). For example, the initial seat layout might look like this:

L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
Otherwise, the seat's state does not change.
Floor (.) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied:

#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
After a second round, the seats with four or more occupied adjacent seats become empty again:

#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
This process continues for three more rounds:

#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##
#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##
#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?
```

In [1]:
#!fsharp
let input = System.IO.File.ReadAllLines "./input.txt"

type Cell =
    | EmptySeat
    | OccupiedSeat
    | Floor

let parseCell = function
    | 'L' -> EmptySeat
    | '#' -> OccupiedSeat
    | '.' -> Floor
    | _ -> failwith "Unexpected input"

let initialGrid = Array.map (Seq.map parseCell) input |> array2D

let neighbors row col grid =
    [ (row - 1, col - 1); (row - 1, col); (row - 1, col + 1);
      (row, col - 1); (row, col + 1);
      (row + 1, col - 1); (row + 1, col); (row + 1, col + 1);
    ] 
    |> List.filter (fun (row, col) -> 
        row >= 0 && row < Array2D.length1 grid &&
        col >= 0 && col < Array2D.length2 grid )
    |> List.map (fun (row, col) -> Array2D.get grid row col )

let occupiedSeatsNearby row col grid =
    neighbors row col grid
    |> List.sumBy (function
        | OccupiedSeat -> 1
        | _ -> 0
    )

let nextState row col (cell: Cell) (grid: Cell[,]) =
    match cell with
    | EmptySeat when (occupiedSeatsNearby row col grid = 0)  -> OccupiedSeat
    | OccupiedSeat when (occupiedSeatsNearby row col grid >= 4) -> EmptySeat
    | x -> x
 

let runTick grid =
    Array2D.mapi (fun row col cell -> nextState row col cell grid) grid

let gridIsEqual (grid1: Cell[,]) (grid2: Cell[,]) =
    seq { for row in 0 .. (Array2D.length1 grid1 - 1) do
            for col in 0 .. (Array2D.length2 grid1 - 1) do
                yield (grid1.[row, col] = grid2.[row, col]) }
    |> Seq.reduce (&&)

let rec runUntilStable (state: Cell[,]) =
    match runTick state with
    | newState when gridIsEqual state newState -> newState
    | newState -> runUntilStable newState

let countOccupied grid =
    let mutable count = 0
    for x in 0 .. (Array2D.length1 grid - 1) do
        for y in 0 .. (Array2D.length2 grid - 1) do
            match grid.[x, y] with
            | OccupiedSeat -> (count <- count + 1)
            | _ -> ()
    count

runUntilStable initialGrid |> countOccupied



2418

```
--- Part Two ---
As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the first seat in each of those eight directions. For example, the empty seat below would see eight occupied seats:

.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:

.............
.L.L.#.#.#.#.
.............
The empty seat below would see no occupied seats:

.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
Also, people seem to be more tolerant than you expected: it now takes five or more visible occupied seats for an occupied seat to become empty (rather than four or more from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Given the same starting layout as above, these new rules cause the seating area to shift around as follows:

L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#
#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count 26 occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?
```

In [1]:
#!fsharp
let firstSeatInEachDirection row col grid =
    let directions = [
        (-1, -1); (-1, 0); (-1, 1);
        (0, -1); (0, 1);
        (1, -1); (1, 0); (1, 1)
    ]

    let rec firstSeatInDirection row col (direction: (int * int)) =
        let rowd, cold = direction
        if row + rowd >= (Array2D.length1 grid) || row + rowd < 0 || col + cold >= (Array2D.length2 grid) || col + cold < 0 then
            Floor
        else
            match Array2D.get grid (row + rowd) (col + cold) with
            | OccupiedSeat -> OccupiedSeat
            | EmptySeat -> EmptySeat
            | Floor -> firstSeatInDirection (row + rowd) (col + cold) direction

    List.map (firstSeatInDirection row col) directions

let countOccupiedSeats =
    List.sumBy (function
    | OccupiedSeat -> 1
    | _ -> 0)

let nextState2 row col cell grid =
    match cell with
    | OccupiedSeat when (firstSeatInEachDirection row col grid |> countOccupiedSeats) >= 5 -> EmptySeat
    | EmptySeat when (firstSeatInEachDirection row col grid |> countOccupiedSeats) = 0 -> OccupiedSeat
    | x -> x

let runTick2 grid =
    Array2D.mapi (fun row col cell -> nextState2 row col cell grid) grid
    
let rec runUntilStable2 state =
    match runTick2 state with
    | newState when gridIsEqual state newState -> newState
    | newState -> runUntilStable2 newState

runUntilStable2 initialGrid |> countOccupied


2144